## ドライブのマウント

In [1]:
# !pip install -qU torch==1.7.1 torchtext==0.8.0 torchvision==0.8.2 torchaudio==0.7.2
# !pip install -q transformers==4.4.2 pytorch_lightning==1.2.1 sentencepiece
# !pip install fugashi
# !pip install ipadic

## 生成器の準備

In [2]:
from tqdm.auto import tqdm
import pandas as pd
from pprint import pprint

INPUT_MAX_LEN = 512  # モデルに入力されるトークン列の最大長。最大長を超えたトークンは切り捨てられる。
OUTPUT_MAX_LEN = 128  # モデルから出力されるトークン列の最大長。最大長を超えないように文が生成されるはず。
AQG_DIR = "../AQG_FINETUNE/model"

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer

# トークナイザー（SentencePiece）
aqg_tokenizer = T5Tokenizer.from_pretrained(AQG_DIR, is_fast=True)

# 学習済みモデル
aqg_model = T5ForConditionalGeneration.from_pretrained(AQG_DIR)

# GPUの利用有無
USE_GPU = torch.cuda.is_available()
if USE_GPU:
    aqg_model.cuda()

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


## 識別器の準備

In [4]:
import pandas as pd
from transformers import AutoTokenizer
import torch
from torch import optim
from torch import nn
import pytorch_lightning as pl

In [5]:
INPUT_MAX_LEN=512
PRETRAINED_MODEL_NAME = "cl-tohoku/bert-base-japanese-char-whole-word-masking"
DIS_CHECKPOINT = "../DIS_FINETUNE/checkpoint/best_loss_epoch=2.ckpt"

In [6]:
class FineTuneBert(pl.LightningModule):
    def __init__(self, model_name_or_path, n_classes, n_epochs=None):
        super().__init__()

        self.model = BertModel.from_pretrained(model_name_or_path, return_dict=True)
        self.classifier = nn.Linear(self.model.config.hidden_size, n_classes)
        self.n_epoch = n_epochs
        self.criterion = nn.CrossEntropyLoss()

        ## BertLayerモジュールの最後を勾配計算ありに変更
        for param in self.model.parameters():
            param.requires_grad = False
        for param in self.model.encoder.layer[-1].parameters():
            param.requires_grad = True

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.model(input_ids, attention_mask=attention_mask)
        preds = self.classifier(output.pooler_output)
        loss = 0
        if labels is not None:
            loss = self.criterion(preds, labels)

        return loss, preds

In [10]:
import torch
from transformers import BertModel, AutoTokenizer
from torchvision import models

# トークナイザー
dis_tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

# 学習済モデル
dis_model = FineTuneBert(PRETRAINED_MODEL_NAME, n_classes=2)
checkpoint = torch.load(DIS_CHECKPOINT, map_location=torch.device('cpu'))
dis_model.load_state_dict(checkpoint["state_dict"])

# GPU利用の有無
USE_GPU = torch.cuda.is_available()
if USE_GPU:
    dis_model.cuda()

## データセット

In [11]:
test_dataset = "./data/test.csv"
question_data_list = []

df = pd.read_csv(test_dataset)

for index, row in df.iterrows():
    question_data_list.append((row["answer"], row["passage"], row["question"]))

## 問題生成

In [15]:
from pytorch_lightning.callbacks import early_stopping
generated_questions = []

dis_model.eval()

for answer, context, example in tqdm(question_data_list):
    # モデルに入力可能な形式に変換する。
    input = f"answer: {answer.lower()} context: {context.lower()}"

    # 入力文をトークナイズする。
    tokenized_inputs = aqg_tokenizer.batch_encode_plus(
        [input], max_length=INPUT_MAX_LEN, truncation=True, 
        padding="longest", return_tensors="pt")

    input_ids = tokenized_inputs['input_ids']
    input_mask = tokenized_inputs['attention_mask']
    if torch.cuda.is_available():
        input_ids = input_ids.cuda()
        input_mask = input_mask.cuda()

    # 問題文を生成する。
    tokenized_outputs = aqg_model.generate(input_ids=input_ids, attention_mask=input_mask, 
        max_length=OUTPUT_MAX_LEN, return_dict_in_generate=True, decoder_start_token_id=0,
        # temperature=1.0,  # 生成にランダム性を入れる温度パラメータ
        num_beams=20,  # ビームサーチの探索幅
        # diversity_penalty=0.1,#1.0,  # 生成結果の多様性を生み出すためのペナルティパラメータ
        # num_beam_groups=4,  # ビームサーチのグループ
        num_return_sequences=20,  # 生成する文の数
        repetition_penalty=10.0, # 同じ文の繰り返し（モード崩壊）へのペナルティ
        # ,early_stopping=True
    )

    # 生成された問題文のトークン列を文字列に変換する。
    outputs = [aqg_tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False) 
        for ids in tokenized_outputs.sequences]

    
    ## 問題の識別
    questions = []
    for question in outputs:
        ## 入力する形式に変換
        input = question + "答えは" + answer

        ## tokenize
        tokenized_input = dis_tokenizer.batch_encode_plus(
            [input],
            max_length=INPUT_MAX_LEN,
            truncation=True,
            add_special_tokens=True,
            padding="longest",
            return_tensors="pt"
        )

        input_ids = tokenized_input["input_ids"]
        input_mask = tokenized_input["attention_mask"]

        ## デバイスの指定
        if torch.cuda.is_available():
            input_ids = input_ids.cuda()
            input_mask = input_mask.cuda()

        ## 識別
        with torch.no_grad():
            _, score = dis_model(input_ids, input_mask)
            questions.append([question, score])
        
    generated_questions.append(questions)

  0%|          | 0/21 [00:00<?, ?it/s]

## 出力&保存

In [16]:
import textwrap

csv_outputs = []

for (answer, context, example), questions in zip(question_data_list, generated_questions):
  print(f"answer: {answer}")
  print("\n".join(textwrap.wrap(f"context:{context}")))
  print(f"example: {example}")
  for question, score in questions:
    pred = torch.argmax(score)
    
    # tf = "○" if pred==1 else "×"

    ## 適切な問題のみ出力
    if pred==1:
      csv_outputs.append([answer, context, question])
      # print(f"  -> {tf} {score} {question}")
      print(f"  -> {question}")
  print()
  
df = pd.DataFrame(csv_outputs, columns=["answer", "passage", "question"])
df.to_csv("./generated/pipline.csv")

answer: 正しい
context:どのようなアルゴリズムでも、処理の流れは、順次、分岐、反復の3つの構造の組み合わせで構成されている。このような処理の流れを制
御構造という。順次とは1つ1つの処理を順番に行うことであり、分岐はある条件に応じて異なる処理を実行することである。また、反復はある条件が満た
されている間はその処理を繰り返し実行することである。
example: 次の文章は正しいか答えなさい。どのようなアルゴリズムでも、処理の流れは、順次、分岐、反復の3つの構造の組み合わせで構成されている。
  -> 次の文章は正しいか答えなさい。順次、分岐、反復の3つの構造の組み合わせで構成されている。
  -> 次の文章は正しいか答えなさい。処理の流れは順次、分岐、反復の3つの構造の組み合わせで構成されている。
  -> 次の文章は正しいですか?処理の流れは順次、分岐、反復の3つの構造の組み合わせで構成されている。
  -> 次の文章は正しいか答えなさい。順次、分岐、反復の3つの構造の組み合わせで構成されている複雑な処理の流れを制御構造という。
  -> 次の文章は正しいか答えなさい。反復はある条件に応じて異なる処理を実行することである。
  -> 次の文章は正しいですか?順次、分岐、反復の3つの構造の組み合わせで構成されている。
  -> 次の文章は正しいか答えなさい。反復とはある条件に応じて異なる処理を実行することである。
  -> 次の文章は正しいか答えよ。順次、分岐、反復の3つの構造の組み合わせで構成されている。
  -> 次の文章は正しいか答えなさい。順次、分岐、反復の3つの構造の組み合わせで構成されており、これらの処理の流れを制御構造という。
  -> 次の文章は正しいですか?処理の流れは順次、分岐、反復の3つの構造で構成されている。
  -> 次の文章は正しいか答えなさい。処理の流れは順次、分岐、反復の3つの構造で構成されており、これらの組み合わせを制御構造という。
  -> 次の文章は正しいか答えなさい。処理の流れは、順次、分岐、反復の3つの構造の組み合わせで構成されている。
  -> 次の文章は正しいか答えなさい。順次、分岐、反復の3つの構造を組み合わせた構成で構成されている。
  -> 次の文章は正しいですか?処理の流れは順次、分岐、反復の3つ